In [2]:
#already built in with python
import os
import datetime 

#installed with conda-forge
import numpy
import xarray as xr
import matplotlib as plt
import folium
import geopandas as gpd
from shapely.geometry import Point

#installed with pip 
import pystac
import pystac_client
import rasterio

# **Same visuals, but different on the inside**
----

The new Earth Observation Processing Framework (**EOPF**) Zarr format is changing the way Sentinel products are delivered. The already .SAFE format will no longer exist to be replaced by Zarr data but don't be scared, Sentinel data is the same, just the storage format and delivery is different. Let's take a look...

## **STAC catalog**
------
The STAC catalog, when you can access the data, has the same apearance and organization schema!

| ![Image 1](img/old.png) | ![Image 2](img/zarr.png) |
|---------------------------------|------------------------|
| CDSE - STAC API                 | EOPF Sentinel Zarr Samples Service STAC API              


But, if you notice carefully, there is a small but significant difference: the EOPF Zarr service has Sentinel-1 SLC data, which didn't exist in the old CDSE STAC catalogue. The main reason for this is the the new Zarr format, because it's cloud-native, can handle in a much simpler way, heavier data, such as Sentinel-1 SLC data (*not only contains the backscatter information, but also the phase informtion, which is essential for InSAR, but we'll talk about that later*)

**Access the data using ID** - The way data is presented is the same, even though the way it is stored is not the same. These are great news! If you want to access a specif product, it still follows the same logic and it did for the CDSE - STAC catalog.

Let's follow this example on how to access Sentinel-1 Level-1 GRD data to see how the data is stored on the EOPF Sentinel Zarr Samples Service.

### 1. Source ID for the STAC

<img src="img/source_id_catalogue.png" width="500"/>

In [3]:
stacID = "eopf-sample-service-stac-api"
stacURL = "https://stac.core.eopf.eodc.eu/"

### 2. Source ID for Sentinel-1 Level-1 GRD

<img src="img/source_id_grd.png" width="500"/>

In [4]:
grdID = "sentinel-1-l1-grd"
grdURL = "https://stac.core.eopf.eodc.eu/collections/sentinel-1-l1-grd"

### 3. Source ID for the specific product

<img src="img/source_id_product.png" width="500"/>

In [5]:
productID = "S1A_IW_GRDH_1SSH_20250708T124813_20250708T124838_059992_0773EF_6FFD"
productURL = "https://stac.core.eopf.eodc.eu/collections/sentinel-1-l1-grd/items/S1A_IW_GRDH_1SSH_20250708T124813_20250708T124838_059992_0773EF_6FFD"

## **But attention** 🚨
---

Two things must be noticed:

**1-** If by any chance you try to look for the same product but on the old catalog we'll find basically the same product but with a different identification name. Are those the same products? Why do they have different names?

**2-** When chekcing for the souce ID for the GRD product, we can see that it's not valid.. What does it mean?

**3** Why can't we download the products like we did on the old STAC catalog?

Let's dive into this problems, one by one!


#### **1. ID names of products**
Let's analyse the ID names of Sentinel products!

Each Sentinel product follows a similar structure, having identifiers to distiguish between products. Each product can be seen a "photo" taken by the satellite and, for each photo, the information is stored under this product ID names.

In [6]:
print("Product ID =", productID)

Product ID = S1A_IW_GRDH_1SSH_20250708T124813_20250708T124838_059992_0773EF_6FFD


And, because this product belongs to a specif group of products (Sentinel-1 Level-1 GRD but it could be, Sentinel-1 Level-1 SLC or Sentinel-1 Level-2A), it also has a collection ID.

In [7]:
print("Collection ID =", grdID)

Collection ID = sentinel-1-l1-grd


In the end, all the products from all the collections are stored on the same STAC catalog.

In [8]:
print("Catalog ID =", stacID)

Catalog ID = eopf-sample-service-stac-api


------

Now let's breakdown the point product name from the EOPF Sentinel Zarr Samples Service: 

1- ```S1A_IW_GRDH_1SSH_20250708T124813_20250708T124838_059992_0773EF_6FFD``` 

and compare it with the same product from the old STAC catalog: 

2- ```S1A_IW_GRDH_1SSH_20250708T124813_20250708T124838_059992_0773EF_E620_COG```

Product from EOPF Sentinel Zarr Samples Service: `S1A_IW_GRDH_1SSH_20250708T124813_20250708T124838_059992_0773EF_6FFD`:

| Part                 | Value               | Meaning                                                                |
|----------------------|---------------------|------------------------------------------------------------------------|
| **S1A**              | S1A                 | **Satellite**: Sentinel-1A                                             |
| **IW**               | IW                  | **Acquisition Mode**: Interferometric Wide Swath                       |
| **GRDH**             | GRDH                | **Product Type**: Ground Range Detected (GRD), High resolution         |
| **1SSH**             | 1SDV                | **Product Level and Polarisation**: Level-1, Dual polarization (VV+VH) |
| **20250708T124813**  | 08-07-2025 12:48:13 | **Start Time** (UTC): acquisition start time                           |
| **20250708T124838**  | 08-07-2025 12:48:38 | **Stop Time** (UTC): acquisition end time                              |
| **059992**           | 059992              | **Absolute Orbit Number**: Sequential number representing how many complete orbits the satellite has made since launch. It increases by 1 for each orbit (approx. every 98 minutes)                          |
| **0773EF**           | 0773EF              | **Mission Data Take ID**: It increases each time the sensor is turned on, kind of identifying a "recording session". It is useful to identify if two products were taken on the same continuous acquisition                                                |
| **6FFD**             | 6FFD                | **Unique Identifier / Product ID**: It is unique for each product, similar to an ID number|

-------

Now, let's compare the two products!

`S1A_IW_GRDH_1SSH_20250708T124813_20250708T124838_059992_0773EF_6FFD` vs `S1A_IW_GRDH_1SSH_20250708T124813_20250708T124838_059992_0773EF_E620_COG`


|      | EOPF Sentinel Zarr Samples Service | old STAC catalog   |  Is it the same?|   
|--------------------------------------|---------------------|--------------------------------------------------------------|-----|
| **Satellite**                        | S1A                 | S1A                                            |✅|
| **Acquisition Mode**                 | IW                  | IW                       |✅|
| **Product Type**                     | GRDH                | GRDH         |✅|
| **Product Level and Polarisation**   | 1SSH                | 1SSH|✅|
| **Start Time**                       | 08-07-2025 12:48:13 |08-07-2025 12:48:13|✅|
| **Stop Time**                        | 08-07-2025 12:48:38 |08-07-2025 12:48:38|✅|
| **Absolute Orbit Number**            | 059992              | 059992                                              |✅|
| **Mission Data Take ID**             | 0773EF              |0773EF                                               |✅|
| **Unique Identifier**                | 6FFD                | E620                                     |❌|
| **Extra Notation**                | -             | COG                                     |❌|

What does this mean? In fact, these are exactly the same products, just the unique identifier and the extra notation is different. This is because:
- The 6FFD unique identifier shows the original ESA-generated product ID, created when the scene was first processed and the E620 identifier was generated when the file was reprocessed and converted into a COG (Cloud Optimized GeoTIFF).
- This leads us to the extra notation found on the old STAC catalog. The COG suffix	indicates the product was repackaged for cloud/web use and, of course, converted into a COG (Cloud Optimized GeoTIFF). We don't face this situation on the new Zarr format.

#### **2. GRD product unvalid**
(no idea why, yet)

(this one: ```S1A_IW_GRDH_1SSH_20250708T124813_20250708T124838_059992_0773EF_6FFD```)

#### **3. Can't douwnload products**
(no idea why, yet)

its not possible to download the files

## **Navigating on the EOPF Zarr STAC**
---
Starting from the begining, let's navigate through the STAC catalog until we find the dataset and products we want - Sentinel-1 GRD!

In [9]:
import requests
from typing import List, Optional, cast
from pystac import Collection, MediaType
from pystac_client import Client, CollectionClient
from datetime import datetime

As a starting point, we need to establish the connection with the EOPF Sentinel Zarr Sample Service STAC Catalog. In fact, we already have the URL link for the catalog, as you can see the following print!

In [10]:
print(stacURL)

https://stac.core.eopf.eodc.eu/


With a function called ``Client.open()`` we can see the organizational tree of the catalog itself! We are on the right path! 

In [11]:
eopf_catalog = Client.open(stacURL) 
eopf_catalog

<Client id=eopf-sample-service-stac-api>

Now that the catalog connection was established we can take a look at all the collections available at the STAC catalog and, hopefully, find the Sentinel-1 GRD! To do so we'll use a function called ``get_all_collections()``, which will list all the collections available on the catalog.

Because the STAC catalog is still under development, not all the collections under the catalog are valid. To avoid getting a long error message we print the collections ID this way, showing only the collections valid and active!

In [12]:
try:
    for collections in eopf_catalog.get_all_collections():
        print(collections.id)
except Exception:
    print(
        "* [https://github.com/EOPF-Sample-Service/eopf-stac/issues/18 where you can check the progress of this issue]"
    )

sentinel-2-l2a
sentinel-3-slstr-l1-rbt
sentinel-3-olci-l2-lfr
sentinel-2-l1c
sentinel-3-slstr-l2-lst
sentinel-1-l1-slc
sentinel-3-olci-l1-efr
sentinel-3-olci-l1-err
sentinel-1-l2-ocn
sentinel-1-l1-grd
* [https://github.com/EOPF-Sample-Service/eopf-stac/issues/18 where you can check the progress of this issue]


Taking a close look we see that Sentinel-1 GRD collection is active and it matches the ID name we store in the beggining.

In [13]:
print("Our collection ID as we stored it before:", grdID)

try:
    for collections in eopf_catalog.get_all_collections():
        if collections.id == grdID:
            print("It's a match!")
except Exception:
    print()

Our collection ID as we stored it before: sentinel-1-l1-grd
It's a match!



Similar to function used before, we'll use the ``get_collection()``function, since we already know the ID of the collection we want to access. After this we'll be able to get more useful information about Sentinel-1 GRD collection! **We are already inside the Sentinel-1 GRD catalog**, now it's time to start looking for useful and meaningful products we can work with.

In [14]:
s1grd = eopf_catalog.get_collection(grdID)
print('ID:              ', s1grd.id)
print('Title:           ', s1grd.title)
print('Description:     ', s1grd.description)
print('License:         ', s1grd.license)
print('Keywords:        ', s1grd.keywords)
print('Providers:       ', s1grd.providers)
print('Extent:          ', s1grd.extent)
print('Links:           ', s1grd.links)
print('Assets:          ', s1grd.assets)


ID:               sentinel-1-l1-grd
Title:            Sentinel-1 Level-1 GRD
Description:      The Sentinel-1 Level-1 Ground Range Detected (GRD) products consist of focused SAR data that has been detected, multi-looked and projected to ground range using the Earth ellipsoid model WGS84. The ellipsoid projection of the GRD products is corrected using the terrain height specified in the product general annotation. The terrain height used varies in azimuth and it is constant in range (For IW/EW modes only the terrain height of first subswath is considered)
License:          proprietary
Keywords:         ['Copernicus', 'Sentinel', 'EU', 'ESA', 'Satellite', 'SAR', 'C-Band', 'GRD']
Providers:        [<pystac.provider.Provider object at 0x160ae47c0>, <pystac.provider.Provider object at 0x160ae4830>, <pystac.provider.Provider object at 0x160ae46e0>]
Extent:           <pystac.collection.Extent object at 0x160271630>
Links:            [<Link rel=items target=https://stac.core.eopf.eodc.eu/colle

##### **summarizing**:
- ``Client.open()``: to access the catalog;
- ``get_all_collections()``: to check all the collection available on the catalog;
- ``get_collection()``: to get a specif collection we want to work with;

## **Searching and filtering inside a EOPF Zarr STAC collection**
---
The next step is to search and look for products inside the collection we chose!

One of the most basic functions we'll use, and probably one of the most important ones is the ``.search()``function. It will allow us to search and filter for collections and products inside a catalog.

We can filter the search on the catalog by using the following elements:
1. **Collections**: using `collections =`, it filters the catalog by the collections, choosing the searching names like **'sentinel-2-l2a'**, **'sentinel-1-l1-grd'** or **'sentinel-1-l1-slc'**;
2. **Date and Time**: using `datetime =`, it filters the catalog by the time interval in which the products can be found. The search should follow this format: **'2020-05-01T00:00:00Z/2025-05-31T23:59:59.999999Z'**, but if you want you can also exclude the time part;
3. **Area of Interest**: using `bbox = ()` and inserting the lat and long values for the top left corner and bottom right corner of the AOI;
4. **IDs**: using `ids =`, it filters the catalog by the source ID of the product, returning only the product matching that ID;

The following function, ``list_found_elements``is pretty useful for the next few steps because it will help us construct a list from any search result we might get.

In [15]:
def list_found_elements(search_result):
    ids = []
    coll = []
    for item in search_result.items(): 
        ids.append(item.id)
        coll.append(item.collection_id)
    return ids , coll

- Search by collection and datetime

In [16]:
search_catalog_collection_datetime = eopf_catalog.search(
    collections = 'sentinel-1-l1-grd', #sentinel-1 GRD collection
    datetime = "2025-07-01T00:00:00Z/2025-07-01T00:15:01Z" #15 minutes of acquisition on 01 July 2025
) 

print("This is how our search literally looks: ", search_catalog_collection_datetime)

collection_datetime_items = list_found_elements(search_catalog_collection_datetime)
print("Found", len(collection_datetime_items[0]), "products on this search.")
print(collection_datetime_items[0])

This is how our search literally looks:  <pystac_client.item_search.ItemSearch object at 0x160222510>
Found 4 products on this search.
['S1A_IW_GRDH_1SDV_20250701T001438_20250701T001503_059883_077026_5239', 'S1A_IW_GRDH_1SDV_20250701T001413_20250701T001438_059883_077026_7438', 'S1A_IW_GRDH_1SDV_20250701T001348_20250701T001413_059883_077026_15F7', 'S1A_IW_GRDH_1SDV_20250701T001319_20250701T001348_059883_077026_3624']


- Search by ID

In [17]:
search_catalog_ID = eopf_catalog.search(
    ids = 'S1A_IW_GRDH_1SDV_20250706T183538_20250706T183603_059967_077303_4A58' 
)

print("This is how our search literally looks: ", search_catalog_ID)

ID_items = list_found_elements(search_catalog_ID)
print("Found", len(ID_items[0]), "product on this search.")
print(ID_items[0])

This is how our search literally looks:  <pystac_client.item_search.ItemSearch object at 0x16021b4d0>
Found 1 product on this search.
['S1A_IW_GRDH_1SDV_20250706T183538_20250706T183603_059967_077303_4A58']


- Search collection, datetime and area of interest

In [18]:
search_catalog_all = eopf_catalog.search(
    collections = 'sentinel-1-l1-grd', #sentinel-1 GRD collection
    datetime = "2025-07-06T00:00:00Z/2025-07-07T23:59:59Z", #acquisitions on the 06 July 2025 and 07 July 2025
    bbox=(-9.283880, 38.634131, #top left corner of AOI
          -8.964918, 38.810976) #bottom right corner of AOI
) 

print("This is how our search literally looks: ", search_catalog_all)

time_items = list_found_elements(search_catalog_all)
print("Found", len(time_items[0]), "products on this search.")
print(time_items[0])

This is how our search literally looks:  <pystac_client.item_search.ItemSearch object at 0x16021b750>
Found 3 products on this search.
['S1A_IW_GRDH_1SDV_20250707T063514_20250707T063539_059974_077342_34A8', 'S1A_IW_GRDH_1SDV_20250706T183538_20250706T183603_059967_077303_4A58', 'S1C_IW_GRDH_1SDV_20250706T064218_20250706T064243_003096_006487_0E3A']


**NOTE**: even though it might seems logical, we can not apply the ``search()``function on a colletion, only on catalogs! Try to uncomment the next cell and you'll see what happens!


In [19]:
#search_s1grd = s1grd.search()

Finally, we need to actually access the items that we've been filtering on the STAC catalog. To do that we need to get their storage location in the cloud, getting its URL link, something similar to his one: `https://objects.eodc.eu/e05ab01a9d56408d82ac32d69a5aae2a:202507-s01siwgrh/06/products/cpm_v256/S1A_IW_GRDH_1SDV_20250706T183538_20250706T183603_059967_077303_4A58.zarr`.

To do so

In [20]:
item_id = ID_items[0][0]

item = s1grd.get_item(item_id)

zarr_assets = item.get_assets(media_type=MediaType.ZARR)

assets_loc = [zarr_assets]

first_item = assets_loc[0]

print('URL for accessing', item_id, 'item:  ', first_item['product'].href)


URL for accessing S1A_IW_GRDH_1SDV_20250706T183538_20250706T183603_059967_077303_4A58 item:   https://objects.eodc.eu:443/e05ab01a9d56408d82ac32d69a5aae2a:202507-s01siwgrh/06/products/cpm_v256/S1A_IW_GRDH_1SDV_20250706T183538_20250706T183603_059967_077303_4A58.zarr


##### **summarizing**:
- ``.search()``: filtering elements inside a catalog;
- ``list_found_elements()``: to list the elements found with the search parameters;
- ``.get_item()``: lolololol;
- ``.get_assets(media_type=MediaType.ZARR)``: ololololl

## **Discovering the groups and sub grouops inside a Zarr product**
----
Now that we've found a specific product we want to work with, let's try to understand how everything is structured inside of it. We'll work with this product: `S1A_IW_GRDH_1SDV_20250706T183538_20250706T183603_059967_077303_4A58`!

The new Zarr format stores Sentinel data in four different groups. It is important to understand this structure because later, when we'll need to access the data, we'll need to know where to find it.

The EOPF Zarr Sentinel structure contains four main groups:
- **Attributes**: STAC format metadata of the imagery, such as chunking information (how the data is divided into the several chunks) and product specific metadata (like acquisition time or sensor specifics);
- **Measurements**: Main retrieved variables, such as reflectance (for Sentinel-1 GRD) and phase information (for Sentinel-1 SLC) or the several band information (when talking about Sentinel-2);
- **Conditions**: Geometric angles, meteorological and instrumental data or any other information concerning;
- **Quality**: Quality information concerning the measurements and masks;

-----

To start accessing and exploring the data we'll need the following libraries:

In [21]:
import os
import xarray as xr

We'll use some funtions from the ``xarray``library. 

The ``open_datatree()`` function allows us to open hierarchical datasets, which is a fancy way to say that our dataset is structured in folders (in our case called groups) and subfolders (or subgroups) - exactly the Zarr format situation. So, in conclusion, using this function will allow us to **load the entire structure of groups and subgroups into accessible objects, where we can navigate, inspect and extract data from any level**.

On the other hand, if we prefer to **open just one group at a time** (intead of unfolding the whole dataset tree), we can use ``open_dataset()``funtion and proceed with the same inspection.

For both functions, there's a few arguments you can input, such as ``engine`` and 

In [22]:
url = "https://objects.eodc.eu/e05ab01a9d56408d82ac32d69a5aae2a:202506-s01siwgrh/29/products/cpm_v256/S1A_IW_GRDH_1SDV_20250629T234043_20250629T234108_059868_076FA4_A8F4.zarr"
urlVH = "https://objects.eodc.eu/e05ab01a9d56408d82ac32d69a5aae2a:202506-s01siwgrh/29/products/cpm_v256/S1A_IW_GRDH_1SDV_20250629T234043_20250629T234108_059868_076FA4_A8F4.zarr/S01SIWGRD_20250629T234043_0025_A342_A8F4_076FA4_VH/measurements"

In [25]:
sample_tree = xr.open_datatree(urlVH,
    engine = "zarr", # storage format
    chunks={}, # allows to open the default chunking
)

sample_set = xr.open_dataset(url,
    engine = "zarr", # storage format
                            )

print(sample_set)

/var/folders/m1/jbwb10r115nghdm3qdhnjtnm0000gn/T/ipykernel_31370/1741898327.py:1: RuntimeWarning: Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  sample_tree = xr.open_datatree(urlVH,


<xarray.Dataset> Size: 0B
Dimensions:  ()
Data variables:
    *empty*
Attributes:
    other_metadata:  {'azimuth_steering_rate': 0.0, 'eopf_category': 'eoconta...
    stac_discovery:  {'assets': {}, 'bbox': [-84.436646, 40.141384, -87.84036...


In [107]:
xr.open_dataset(url, engine="zarr")
#xr.open_zarr(url, chunks={})


<xarray.Dataset> Size: 0B
Dimensions:  ()
Data variables:
    *empty*
Attributes: (2)

In [29]:
import zarr
zarr_store = zarr.open(url, mode='r')
print(zarr_store.tree())

/


In [118]:
#print(sample_set.data_vars)
print(sample_set["grd"])


KeyError: "No variable named 'grd'. Variables on the dataset include ['VH_conditions_antenna_pattern_elevation_angle', 'VH_conditions_antenna_pattern_incidence_angle', 'VH_conditions_antenna_pattern_roll', 'VH_conditions_antenna_pattern_terrain_height', 'VH_conditions_antenna_pattern_azimuth_time', ..., 'VV_quality_calibration_line', 'VV_quality_calibration_pixel', 'VV_quality_noise_noise_power_correction_factor', 'VV_quality_noise_number_of_noise_lines', 'VV_quality_noise_azimuth_time']"

In [56]:
for i in sample_tree.groups:
    print(i)

#for name, child in sample_tree.children.items():
#    print(name)


/
/S01SIWGRD_20250629T234043_0025_A342_A8F4_076FA4_VH
/S01SIWGRD_20250629T234043_0025_A342_A8F4_076FA4_VV
/S01SIWGRD_20250629T234043_0025_A342_A8F4_076FA4_VH/conditions
/S01SIWGRD_20250629T234043_0025_A342_A8F4_076FA4_VH/measurements
/S01SIWGRD_20250629T234043_0025_A342_A8F4_076FA4_VH/quality
/S01SIWGRD_20250629T234043_0025_A342_A8F4_076FA4_VV/conditions
/S01SIWGRD_20250629T234043_0025_A342_A8F4_076FA4_VV/measurements
/S01SIWGRD_20250629T234043_0025_A342_A8F4_076FA4_VV/quality
/S01SIWGRD_20250629T234043_0025_A342_A8F4_076FA4_VH/conditions/antenna_pattern
/S01SIWGRD_20250629T234043_0025_A342_A8F4_076FA4_VH/conditions/attitude
/S01SIWGRD_20250629T234043_0025_A342_A8F4_076FA4_VH/conditions/azimuth_fm_rate
/S01SIWGRD_20250629T234043_0025_A342_A8F4_076FA4_VH/conditions/coordinate_conversion
/S01SIWGRD_20250629T234043_0025_A342_A8F4_076FA4_VH/conditions/doppler_centroid
/S01SIWGRD_20250629T234043_0025_A342_A8F4_076FA4_VH/conditions/gcp
/S01SIWGRD_20250629T234043_0025_A342_A8F4_076FA4_VH/cond

**Can't use analysis ``op_mode``**: this means that only the ``op_mode = "native"``works. This is specific for Sentinel-1, because if you try the same with Sentinel-2 you don't face the same problem, as you can see bellow.

In [12]:
url_s2 = "https://objects.eodc.eu/e05ab01a9d56408d82ac32d69a5aae2a:202507-s02msil2a/10/products/cpm_v256/S2B_MSIL2A_20250710T231559_N0511_R087_T03WVS_20250710T233659.zarr"

sample_tree = xr.open_datatree(url_s2,
    engine = "zarr", # storage format
    op_mode = "analysis", # no analysis mode
    product_type = "MSIL2A",
    chunks={}, # allows to open the default chunking
)

/opt/anaconda3/envs/eopf_toolkit/lib/python3.13/site-packages/xarray_eopf/amodes/sentinel2.py:116: UserWarning: Analysis mode not implemented for given source, return data tree as-is.
  warnings.warn(


In [89]:
sample_set = xr.open_dataset(url_s2,
    engine = "zarr", # storage format
    op_mode = "native", # no analysis mode
    #product_type = "MSIL2A",
    #product_type = "S01SIWGRH",
    #product_type = "SIWGRH",
    chunks={}, # allows to open the default chunking
)

In [90]:
print(sample_tree)

<xarray.DataTree>
Group: /
│   Attributes: (2)
├── Group: /S01SIWGRD_20250629T234043_0025_A342_A8F4_076FA4_VH
│   │   Attributes: (2)
│   ├── Group: /S01SIWGRD_20250629T234043_0025_A342_A8F4_076FA4_VH/conditions
│   │   ├── Group: /S01SIWGRD_20250629T234043_0025_A342_A8F4_076FA4_VH/conditions/antenna_pattern
│   │   │       Dimensions:           (azimuth_time: 27, slant_range_time: 692)
│   │   │       Coordinates:
│   │   │         * azimuth_time      (azimuth_time) datetime64[ns] 216B 2025-06-29T23:40:44....
│   │   │           slant_range_time  (azimuth_time, slant_range_time) float64 149kB dask.array<chunksize=(27, 692), meta=np.ndarray>
│   │   │       Data variables:
│   │   │           elevation_angle   (azimuth_time, slant_range_time) float32 75kB dask.array<chunksize=(27, 692), meta=np.ndarray>
│   │   │           incidence_angle   (azimuth_time, slant_range_time) float32 75kB dask.array<chunksize=(27, 692), meta=np.ndarray>
│   │   │           roll              (azimuth_time)

In [30]:
def print_gen_structure(node, indent=""):
    print(f"{indent}{node.name}")     #allows us access each node
    for child_name, child_node in node.children.items(): #loops inside the selected nodes to extract naming
        print_gen_structure(child_node, indent + "  ") # prints the name of the selected nodes

In [31]:
print_gen_structure(sample.root)

None
  S01SIWGRD_20250629T234043_0025_A342_A8F4_076FA4_VH
    conditions
      antenna_pattern
      attitude
      azimuth_fm_rate
      coordinate_conversion
      doppler_centroid
      gcp
      orbit
      reference_replica
      replica
      terrain_height
    measurements
    quality
      calibration
      noise
  S01SIWGRD_20250629T234043_0025_A342_A8F4_076FA4_VV
    conditions
      antenna_pattern
      attitude
      azimuth_fm_rate
      coordinate_conversion
      doppler_centroid
      gcp
      orbit
      reference_replica
      replica
      terrain_height
    measurements
    quality
      calibration
      noise


In [149]:
import requests
from typing import List, Optional, cast
from pystac import Collection, MediaType
from pystac_client import Client, CollectionClient
from datetime import datetime